In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import urllib
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

In [3]:
URL='http://selene.bu.uni.wroc.pl:8000/publikacje/ItemsList.php?Ind=7&Str=First&Item=Instytut%20Informatyki&Wydz=0&Ekr=0&GrTyp=0'

In [4]:
def get_page(page):
    return urllib.request.urlopen(page).read()

def normalize_name(name): # Imię 2Imię Nazwisko -> I. Nazwisko
    names = name.split(' ')
    return '{}. {}'.format(names[0][0], names[-1])

In [5]:
page=get_page(URL)

In [6]:
code = BeautifulSoup(page, "lxml")

In [7]:
pub_table = code.find('table', attrs={'class':'Kk'})
pubs = []
for pub in pub_table.find_all("tr", recursive=False):
    ii_authors = pub.find('td', attrs={'class': 'KK-HG'}).string
    descr_td = pub.find('td', attrs={'class': 'KK-1'})
    descr = descr_td.contents
    info = descr_td.find_next('td', attrs={'class': 'KK-1'})
    ii_authors = [a.strip() for a in ii_authors.split(',')] # lista autorów z ii
    ii_authors = [' '.join(a.split(' ')[:2]) for a in ii_authors] # Usuń 2 imiona
    pubs += [(ii_authors, descr, info)]
print("Liczba publikacji: ", len(pubs))

Liczba publikacji:  85


In [8]:
all_ii_authors = set()
for authors, descr, info in pubs:
    all_ii_authors |= set(authors)

In [9]:
publications = []

for iiauthors, descr, info in pubs:
    #authors = ', '.join([to_norm_name(a) for a in authors])
    title, authors = descr[0].split(' / ')
    bib_index = authors.find('Bibliogr')
    if bib_index >= 0:
        authors = authors[:bib_index]
    authors = [normalize_name(a) for a in authors.split(', ')]
    authors = ', '.join(authors)
    
    source = descr[2]
    bib_index = source.find(': il.')
    if bib_index >= 0:
        source = source[:bib_index]
    if source.startswith('// '):
        source = source[3:]
    if source.startswith('W: '):
        source = source[3:]

    abstract = None
    try:
        abstract = info.find('a').text
    except AttributeError:
        pass
    
    publications.append((
        set(iiauthors),
        authors,
        title,
        source,
        abstract
    ))

In [10]:
SAM='Samodzielny'
PGK='Pracownia Grafiki Komputerowej'
ZIOB='Zakład Inteligencji Obliczeniowej'
ZIO='Zakład Inżynierii Oprogramowania'
ZJP='Zakład Języków Programowania'
ZMN='Zakład Metod Numerycznych'
ZOK='Zakład Optymalizacji Kombinatorycznej'
ZTI='Zakład Teorii Informatyki i Baz Danych'
ZZOIA='Zakład Złożoności Obliczeniowej i Algorytmów'
STU='STUDENT'
UNK='UNK'

authors_map = {
 'Bartkowiak Anna' : ZMN,
 'Bednarczyk Bartosz' : ZJP,
 'Biernacki Dariusz' : ZJP,
 'Bieńkowski Marcin' : ZOK,
 'Byrka Jarosław' : ZOK,
 'Charatonik Witold' : ZJP,
 'Chorowski Jan' : ZIOB,
 'Chudy Filip' : ZMN,
 'Danielski Daniel': STU,
 'Derkowski Piotr': STU,
 'Dudek Bartłomiej': ZZOIA,
 'Dudycz Szymon' : ZOK,
 'Ferens Robert': ZZOIA,
 'Garncarek Paweł': ZZOIA,
 'Gawrychowski Paweł': ZZOIA,
 'Gańczorz Michał': ZZOIA,
 'Ghosal Pratik' : ZOK,
 'Głuch Grzegorz': STU,
 'Jeż Artur': ZZOIA,
 'Jeż Łukasz' : ZOK,
 'Jurdziński Tomasz': ZZOIA,
 'Karpiński Michał': ZZOIA,
 'Kieroński Emanuel': ZTI,
 'Kisielewicz Andrzej': ZZOIA,
 'Kowalski Jakub' : ZIOB,
 'Kraska Artur': ZOK,
 'Kunysz Adam' : ZOK,
 'Laskoś-Grabowski Paweł': ZOK,
 'Lewandowski Mateusz' : ZOK,
 'Liu Hsiang-Hsuan' : ZOK,
 'Loryś Krzysztof': ZZOIA,
 'Malik Szymon': STU,
 'Marcinkowski Jan' : ZOK,
 'Marcinkowski Jerzy': ZTI,
 'Michaliszyn Jakub': ZTI,
 'Miernik Radosław': STU,
 'Nowak Rafał' : ZMN,
 'Nowicki Krzysztof': ZZOIA,
 'Ostropolski-Nalewaja Piotr': ZTI,
 'Otop Jan': ZTI,
 'Paluch Katarzyna' : ZOK,
 'Pawlikowski Maciej': STU,
 'Piecuch Krzysztof' : ZOK,
 'Piróg Maciej' : ZJP,
 'Polesiuk Piotr' : ZJP,
 'Pytlik Piotr': STU,
 'Rajba Paweł' : ZMN,
 'Rybicki Bartosz': ZOK,
 'Rychlikowski Paweł' : ZIOB,
 'Różański Michał': ZZOIA,
 'Schmidt Paweł' : ZOK,
 'Sieczkowski Filip' : ZJP,
 'Siwek Cezary': STU,
 'Sornat Krzysztof' : ZOK,
 'Spoerhase Joachim': ZOK,
 'Spyra Aleksandra' : STU,
 'Stachowiak Grzegorz': ZZOIA,
 'Szykuła Marek': ZZOIA,
 'Sękowski Jakub': STU,
 'Wieczorek Paweł' : ZJP,
 'Wieczorek Piotr': ZTI,
 'Witkowski Piotr': ZJP,
 'Wnuk-Lipiński Piotr' : ZIOB,
 'Woźny Paweł' : ZMN,
 'Zykowski Paweł': STU,
 'Łańcucki Adrian' : ZIOB,
 'Łopuszański Jakub': ZZOIA,
 'Żarczyński Łukasz': ZZOIA}

In [11]:
group_map = {}
for k,v in authors_map.items():
    if v not in group_map:
        group_map[v] = set()
    group_map[v].add(k)

In [12]:
displ = ''
for k, v in group_map.items():
    displ += '<h3>{}</h3><ul>'.format(k)
    for author in sorted(v):
        displ += '<li>{}</li>'.format(author)
    displ += '</ul>'

In [13]:
display(HTML(displ))

In [14]:
SEC=ZIOB
html = ''
for SEC in group_map.keys():
    html += '<h1>{}</h1>'.format(SEC)
    html += '<ol>'
    for pub in publications:
        if len(group_map[SEC] & pub[0]) > 0:
            html += '<li>{authors}, <em>{title}</em>, W: {source}'.format(
                authors=pub[1],
                title=pub[2],
                source=pub[3]
            )
            if pub[4] is not None:
                html += ' (<a href={url}>{url}</a>)'.format(url=pub[4])
            html += '.'
    html += '</ol>'

In [15]:
html = (html
        .replace("~''a", 'á')
        .replace("~:u", "ü")
        .replace("U+0026amp;amp;", "and"))

In [16]:
display(HTML(html))